<a href="https://colab.research.google.com/github/AmrAlaaFCIH/text_summarization/blob/main/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
cd /content/drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [4]:
!pip3 install transformers==4.7.0 nltk==3.5 protobuf==3.15.3 sentencepiece==0.1.96

     |████████████████████████████████| 2.5 MB 27.2 MB/s 
     |████████████████████████████████| 1.4 MB 64.5 MB/s 
     |████████████████████████████████| 1.0 MB 47.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.7 MB/s 
     |████████████████████████████████| 880 kB 41.2 MB/s 
     |████████████████████████████████| 3.3 MB 52.0 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434692 sha256=e57a46cf5f444f406a2dbd97d24777024efb0bd1997411cb82e9f22bd93d1ad7
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d1443b44351ab220c949486d7fb43b11536182ec9dae96f7f0eec89b5f239f50
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built nltk sacremoses
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstallin

In [5]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Loading our Data set


In [6]:
data = pd.read_csv('Final_dataset.csv', index_col=0)
data.head(10)

,description,content
0,تجددت المعارك اليوم الجمعة بمحيط العاصمة الليب...,تجددت المعارك اليوم الجمعة بمحيط العاصمة الليب...
1,كشف الفنان والمقاول المصري المعارض محمد علي عن...,عبد الرحمن محمد-الجزيرة نت كشف الفنان والمقاول...
2,شهدت تركيا اليوم الجمعة الإعلان رسميا عن ميلاد...,خليل مبروك-إسطنبول شهدت تركيا اليوم الجمعة الإ...
3,عززت السلطات الهندية اليوم الجمعة إجراءات الأم...,عززت السلطات الهندية اليوم الجمعة إجراءات الأم...
4,احتشد آلاف المسلمين في عدد من الدول الإسلامية ...,احتشد آلاف المسلمين في عدد من الدول الإسلامية ...
5,اصطدم محمد أبو ركبة بعد حصوله على الدكتوراه با...,"رائد موسى-غزة ""أصبحت مدمنا على متابعة كرة القد..."
6,يعد فيلم البابوان، المعروض على نتفلكس، واحدا م...,سارة عابدين يعد فيلم البابوان، الذي عرضته منصة...
7,قال السفير الليبي في أنقرة إن بلاده تحتاج دعما...,قال السفير الليبي في تركيا عبد الرزاق مختار عب...
8,"حمية ""الديتوكس"" موضة غذائية انتشرت سريعا حول ا...","فريدة أحمد حمية ""الديتوكس"" موضة غذائية انتشرت ..."
9,ببدلته الحمراء ولحيته الكثيفة البيضاء والهدايا...,ببدلته الحمراء ولحيته الكثيفة البيضاء وكيسه ال...


In [8]:
data["content"]=data["content"].str.replace('\s{2,}|[0-9]{5,}|[A-Za-z]+[0-9a-zA-z|-]+','',regex=True).str.replace('\n',' ')

In [ ]:
data.shape

(5870, 2)

###Prepare our Model

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
m_name = "marefa-nlp/summarization-arabic-english-news"

tokenizer = AutoTokenizer.from_pretrained(m_name)
model = AutoModelWithLMHead.from_pretrained(m_name).to(device)

Downloading:   0%|          | 0.00/765 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/847k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:847: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/3.16G [00:00<?, ?B/s]

In [10]:
def get_summary(text, tokenizer, model, device="cpu", num_beams=2):
    if len(text.strip()) < 50:
        return ["Please provide a longer text"]
    
    text = "summarize: <paragraph> " + " <paragraph> ".join([ s.strip() for s in sent_tokenize(text) if s.strip() != ""]) + " </s>"
    #print(text)
    text = text.strip().replace("\n","")
    #print(text)
    
    tokenized_text = tokenizer.encode(text, return_tensors="pt").to(device)
    #print(tokenized_text)
    
    summary_ids = model.generate(
            tokenized_text,
            max_length=512,
            num_beams=num_beams,
            repetition_penalty=1.5, 
            length_penalty=1.0, 
            early_stopping=True
    )
    
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return [ s.strip() for s in output.split("<hl>") if s.strip() != "" ]

## Predicting using our model

In [ ]:
for i in data['content'].tolist():
  print("Original Article:", i)
  print("\n===========\nSummary: \n")
  hls = get_summary(i, tokenizer, model, device)
  for hl in hls:
    print("\t-", hl)
  print("--------------------------------------------------------------------------------------")

Original Article: تجددت المعارك اليوم الجمعة بمحيط العاصمة الليبية طرابلس، وأفادت التقارير بتراجع قوات اللواء المتقاعد خليفة حفتر ومقتل أحد أبرز قادتها، في حين أكد الجيش التركي جاهزيته للتوجه لليبيا فور تلقيه التعليمات. وقد أفاد مصدر عسكري من حكومة الوفاق بتجدد الاشتباكات المسلحة بأغلب محاور القتال جنوبي طرابلس. واستعادت قوات الوفاق مواقع خسرتها في وقت سابق، وأجبرت قوات حفتر على الانسحاب إلى خلف معسكر اليرموك بمحيط طرابلس. وذكرت مصادر من قوات الوفاق أن مقاتلي الخطوط الأمامية في قوات حفتر كانوا من مسلحي شركة "فاغنر" الروسية، الذين ينفذون عمليات قتالية ضد قوات حكومة الوفاق باستخدام أسلحة نوعية حديثة. وأكدت وسائل إعلام موالية لقوات حفتر أنها فقدت قائدا عسكريا بارزا يحمل رتبة عقيد ركن ويدعى فتحي محمد المسلاتي. ولقي المسلاتي مصرعه اليوم الجمعة بقصف صاروخي في محور اليرموك جنوب العاصمة طرابلس. وكان المسلاتي يقود سلاح الدبابات والمدرعات باللواء 106 التابع لقوات حفتر. في السياق ذاته، استهدفت طائرات حفتر اليوم الجمعة تمركزات عسكرية بمحيط مصفاة الزاوية شمال غربي البلاد. وكانت المدينة قد تعرضت أمس

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


	- وتفيد التقارير بأن قوات حفتر تراجعت إلى خلف معسكر اليرموك بمحيط طرابلس.
	- وقد استهدفت قوات حفتر تمركزات عسكرية بمحيط مصفاة الزاوية.
	- وتقول المصادر إن القوات المسلحة التركية مستعدة للتوجه إلى ليبيا فور تلقيها التعليمات.
--------------------------------------------------------------------------------------
Original Article: عبد الرحمن محمد-الجزيرة نت كشف الفنان والمقاول المصري المعارض محمد علي عن وثيقة التوافق المصري التي أعلن في مؤتمر لندن يوم 20 نوفمبر/تشرين الثاني الماضي عزمه إعدادها مع قوى وطنية، تمهيدا لإسقاط نظام الرئيس الحالي عبد الفتاح السيسي. وقال علي في مقطع فيديو نشره على حساباته بمواقع التواصل الاجتماعي، إنه التقى ممثلين عن كافة التيارات السياسية والفكرية التي وجدها في "حالة إنهاك شديد بسبب استهداف النظام لها بشكل مستمر". وأشار إلى أنه عمل مع مختلف التيارات على إيجاد نقاط عمل مشتركة يتجنبون بها مواطن الخلاف، للعمل على إنقاذ مصر وعودة الحرية والكرامة والعدالة. وابتدأ محمد علي حديثه بالتأكيد أن ما حدث من الرئيس الحالي السيسي في يوليو/تموز 2013 هو "انقلاب عسكري ضد رئيس منت